In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from sklearn.metrics import f1_score, recall_score

In [2]:
def init_spark():
    spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()
    return spark
spark_object = init_spark()

23/03/15 18:29:55 WARN Utils: Your hostname, karyubuntu resolves to a loopback address: 127.0.1.1; using 172.31.170.249 instead (on interface wlp0s20f3)
23/03/15 18:29:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/15 18:29:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
train_spark = spark_object.read.option("delimiter", ",").csv("./data/trainbalanced.csv", header='true', inferSchema='true')

In [4]:
for i in train_spark.columns:
    oldname = i
    if "." in oldname:
        newname = oldname.replace(".","")
        train_spark = train_spark.withColumnRenamed(oldname, newname)

In [5]:
train_data, test_data = train_spark.randomSplit([0.7, 0.3], seed=123)

In [6]:
input_list =[]
for i in train_spark.columns:
    if i != 'isFraud':
        input_list.append(i)

In [7]:
assembler = VectorAssembler(inputCols=input_list,outputCol='features')

In [8]:
train_features = assembler.transform(train_data)
test_features = assembler.transform(test_data)

In [9]:
try:
    dt = RandomForestClassifier(featuresCol='features', labelCol='isFraud')
    model = dt.fit(train_features)
except Exception as e:
    print(e)

23/03/15 18:30:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [10]:
predictions = model.transform(test_features)
# evaluator = BinaryClassificationEvaluator(labelCol='isFraud', rawPredictionCol='prediction', metricName='areaUnderROC')


In [11]:
y_pred=predictions.select("prediction").collect()
y_orig=predictions.select("isFraud").collect()

In [13]:
print(f1_score(y_orig, y_pred))
print(recall_score(y_orig, y_pred))

0.9151529087146472
0.9046825276557054


In [11]:
# rmse = evaluator.evaluate(predictions)
# print('RoC =', rmse)

RoC = 0.9224618718433263


In [12]:
# test_spark = spark_object.read.option("delimiter", ",").csv("./data/testbalanced.csv", header='true', inferSchema='true')
# num_rows = test_spark.count()
# num_cols = len(test_spark.columns)
# print("Shape of DataFrame: ({}, {})".format(num_rows, num_cols))

# for i in test_spark.columns:
#     oldname = i
#     if "." in oldname:
#         newname = oldname.replace(".","")
#         test_spark = test_spark.withColumnRenamed(oldname, newname)

In [13]:
# assembler = VectorAssembler(inputCols=input_list,outputCol='features')
# test_data = assembler.transform(test_spark)

In [14]:
# predictions = model.transform(test_data)